In [0]:
from pyspark.sql.functions import *

symptoms_question = spark.read.table("surakshadb.bronze_stg.symptoms_raw")\
    .select(col("name").alias("name"),
            col("type").alias("type"),
            col("text").alias("question1"),
            col("laytext").alias("question2"),
            col("min").alias("min_value"),
            col("max").alias("max_value"),
            col("default").alias("default_value"),
            col("category").alias("category"),
            col("alias").alias("alias"),
            col("wiki").alias("wiki"),
            col("wiki2").alias("wiki2"),
            col("wiki3").alias("wiki3"),
            col("wiki4").alias("wiki4"),
            col("subcategory1").alias("subcategory1"),
            col("subcategory2").alias("subcategory2"),
            col("subcategory3").alias("subcategory3"),
            col("subcategory4").alias("subcategory4"),
            col("IsPatientProvided").alias("IsPatientProvided"),
            col("IsInvasive").alias("IsInvasive")           
            )

In [0]:
symptoms_question.limit(3).display()

name,type,question1,question2,min_value,max_value,default_value,category,alias,wiki,wiki2,wiki3,wiki4,subcategory1,subcategory2,subcategory3,subcategory4,IsPatientProvided,IsInvasive
Age,integer,What is the age?,Age,18.0,120.0,35.0,Constitutional and vital signs physical examination,,,,,,,,,,true,false
BMI,double,What is the BMI?,"What is your BMI (body mass index, calculate by either of these formulas BMI = 703 ? weight (lbs) / [height (in)]2 or BMI = weight (kg) / [height (m)]2)",11.0,60.0,19.0,Constitutional and vital signs physical examination,"weight, height",,,,,,,,,false,null
Temp,double,"What is the highest tympanic (ear) temperature measured (Fahrenheit), which is usually higher by 1F than axilar or forehead temperature?","What is the highest recorded temperature (using the ear thermometer)? Hint: the ear temperature is usually 1F higher than temperature measured on forehead or armpit temperature, so to calculate the ear temperature add 1F to the temperature measured on forehead or in armpit)",95.0,109.0,98.0,Constitutional and vital signs physical examination,,Temp website,Temp website 2,Temp website 3,Temp website 4,,,,,false,null
HeartRate,integer,What is the pulse rate (not always equals the heart rate)?,What is your current pulse rate (Hint: the pulse rate is the count of pulse beats within the minute)?,0.0,220.0,70.0,Constitutional and vital signs physical examination,,,,,,,,,,false,null
SBP,integer,What is the systolic blood pressure?,"What is your current systolic blood pressure? (Hint: the top number from your blood pressure reading is called the ""systolic blood pressure"", for example if your meter says ""120/80 mmHg"", the systolic blood pressure is ""120"")",0.0,300.0,115.0,Constitutional and vital signs physical examination,"systolic, sbp",,,,,,,,,false,null


In [0]:
symptoms_choices = spark.read.table("surakshadb.bronze_stg.symptoms_raw")\
    .filter(col("type") == "categorical")\
        .withColumn("choices", explode(col("choices")))\
            .select(
                col("name").alias("name"),
                col("type").alias("type"),
                col("choices.value").alias("choice_value"),
                col("choices.text").alias("choice_text"),
                col("choices.laytext").alias("choice_laytext"),
                col("choices.relatedanswertag").alias("answertag")
                )

In [0]:
symptoms_choices.limit(3).display()

name,name,choice_value,choice_text,choice_laytext,answertag
EdemaRos,categorical,1,Data unavailable (i.e. unable to assess).,Skip this question.,null
EdemaRos,categorical,2,No edema.,No swelling on lower extremities.,No swelling on lower extremities.
EdemaRos,categorical,3,Yes. On one side only (i.e. on right lower extremity only).,"Yes. On one side only (i.e. on right ankle, calf).","Yes. On one side only (i.e. on right ankle, calf)."
EdemaRos,categorical,4,Yes. On both sides. (i.e. both lower extremities - symmetric or close to symmetric).,Yes. On both sides. (i.e. both lower legs - symmetric or close to symmetric).,Yes. On both sides. (i.e. both lower legs - symmetric or close to symmetric).
ElevatedSystolicBp,categorical,1,Data unavailable (i.e. unable to assess).,Skip this question.,null
ElevatedSystolicBp,categorical,2,No.,No.,properly checked systolic blood pressure of less than 130mmHg
ElevatedSystolicBp,categorical,3,Yes.,Yes.,null
ElevatedDiastolicBp,categorical,1,Data unavailable (i.e. unable to assess).,Skip this question.,null
ElevatedDiastolicBp,categorical,2,No.,No.,properly checked diastolic blood pressure of less than 80mmHg
ElevatedDiastolicBp,categorical,3,Yes.,Yes.,null


In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("surakshadb.silver_stg.symptoms_question"):
    deltaTable = DeltaTable.forName(spark, "surakshadb.silver_stg.symptoms_question")

    deltaTable.alias("target")\
        .merge(
            symptoms_question.alias("source"),
            "target.name = source.name AND\
            target.type = source.type"
        ).whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()
else:
    symptoms_question.write.format("delta").saveAsTable("suraldb.silver_stg.symptoms_question")


In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("surakshadb.silver_stg.symptoms_question"):
    deltaTable = DeltaTable.forName(spark, "surakshadb.silver_stg.symptoms_question")

    deltaTable.alias("target")\
        .merge(
            symptoms_choices.alias("source"),
            "target.name = source.name AND\
                target.type = source.type"
        ).whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
                .execute()
else:
    symptoms_choices.write.format("delta").saveAsTable("surakshadb.silver_stg.symptoms_question")